In [132]:
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter, resample

In [133]:
def _butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=1):
    b, a = _butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [134]:
filename = 'SIV101 HIP (2019-08-01)RAW_0.csv'

In [135]:
# Sample rate and desired cutoff frequencies (in Hz).
original_fz = 100.0

In [136]:
df = pd.read_csv(filename, skiprows=10)[['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z']]
df.head()

,Accelerometer X,Accelerometer Y,Accelerometer Z
0,0.031,1.047,0.105
1,0.027,1.031,0.102
2,0.023,1.027,0.098
3,0.023,1.023,0.098
4,0.027,1.020,0.098


In [137]:
df.columns = ['X', 'Y', 'Z']
df.shape

(8639990, 3)

#### Make shape a divisible of 10

In [138]:
df = df[:-(df.shape[0] % 10)] if df.shape[0] % 10 != 0 else df[:]

In [139]:
df.shape

(8639990, 3)

### Step 1: Aliasing Filter (use Butterworth Filter)

In [140]:
lowcut = 0.01
highcut = 7
order = 1

In [141]:
df['X-s1'] = butter_bandpass_filter(df['X'], lowcut, highcut, original_fz, order)
df['Y-s1'] = butter_bandpass_filter(df['Y'], lowcut, highcut, original_fz, order)
df['Z-s1'] = butter_bandpass_filter(df['Z'], lowcut, highcut, original_fz, order)

In [142]:
df.shape

(8639990, 6)

### Step 2: Actigraph Filter (use Butterworth filter)

In [143]:
lowcut = 0.29
highcut = 1.63
order = 1

In [144]:
df['X-s2'] = butter_bandpass_filter(df['X-s1'], lowcut, highcut, original_fz, order)
df['Y-s2'] = butter_bandpass_filter(df['Y-s1'], lowcut, highcut, original_fz, order)
df['Z-s2'] = butter_bandpass_filter(df['Z-s1'], lowcut, highcut, original_fz, order)

### Step 3: Downsample

In [145]:
downsample_size = int(df.shape[0]/10)

In [146]:
df.shape[0]

8639990

In [147]:
Xs3 = resample(df['X-s2'], downsample_size)
Ys3 = resample(df['Y-s2'], downsample_size)
Zs3 = resample(df['Z-s2'], downsample_size)

In [148]:
Xs3.shape

(863999,)

### Step 4: Truncate [-2.13, 2.13]

In [149]:
Xs4 = np.clip(Xs3, a_min=-2.13, a_max=2.13)
Ys4 = np.clip(Ys3, a_min=-2.13, a_max=2.13)
Zs4 = np.clip(Zs3, a_min=-2.13, a_max=2.13)

### Step 5: Rectification

In [150]:
Xs5 = np.absolute(Xs4)
Ys5 = np.absolute(Ys4)
Zs5 = np.absolute(Zs4)

### Step 6: Deadband (Rectifiy at 0.068)

In [151]:
Xs5[Xs5 < 0.068] = 0.
Xs6 = Xs5
Ys5[Ys5 < 0.068] = 0.
Ys6 = Ys5
Zs5[Zs5 < 0.068] = 0.
Zs6 = Zs5

### Step 7: Convert to 8 bit resolution

In [152]:
Xs7 = Xs6 * (128/2.13)
Ys7 = Ys6 * (128/2.13)
Zs7 = Zs6 * (128/2.13)

### Step 8: Accumulate to epoch

In [153]:
req_epoch = 15  #seconds
mul_epoch = 10 * req_epoch  

In [154]:
mul_epoch

150

In [155]:
len(Xs7)

863999

In [156]:
Xs8 = Xs7[:-(len(Xs7) % mul_epoch)] if len(Xs7) % mul_epoch != 0 else Xs7
Ys8 = Ys7[:-(len(Ys7) % mul_epoch)] if len(Ys7) % mul_epoch != 0 else Ys7
Zs8 = Zs7[:-(len(Zs7) % mul_epoch)] if len(Zs7) % mul_epoch != 0 else Zs7

In [157]:
Xs8.shape, Ys8.shape, Zs8.shape

((863850,), (863850,), (863850,))

In [158]:
X_out = np.sum(Xs8.reshape(-1, mul_epoch), axis=1)
Y_out = np.sum(Ys8.reshape(-1, mul_epoch), axis=1)
Z_out = np.sum(Zs8.reshape(-1, mul_epoch), axis=1)

In [159]:
df_out = pd.DataFrame(X_out, index=range(0, len(X_out)))
df_out.columns = ['X_out']
df_out['Y_out'] = Y_out
df_out['Z_out'] = Z_out

In [160]:
df_out

,X_out,Y_out,Z_out
0,0.000000,249.612983,0.000000
1,78.979873,31.471054,46.913348
2,0.000000,8.771871,87.323216
3,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000
...,...,...,...
5754,0.000000,0.000000,4.579817
5755,0.000000,0.000000,0.000000
5756,0.000000,0.000000,0.000000
5757,10.445975,0.000000,0.000000


### Save results

In [161]:
df_out.to_csv(filename.replace('.csv', '_filt.csv'), index=None)